In [1]:
import os
import sys

import numpy as np

import pandas as pd
import glob

In [2]:
BASE_MODEL_PATH="trained_model"
%mkdir -p $BASE_MODEL_PATH

In [3]:
SAVE_MODEL_PATH="{}/simple_classification".format(BASE_MODEL_PATH)
%mkdir -p $SAVE_MODEL_PATH

## Data preparation

In [4]:
from models.modelutils import dir2filedict, split_fdict

Using TensorFlow backend.


In [5]:
fdict = dir2filedict("data")

In [6]:
categories = sorted(fdict.keys())

In [7]:
trdict, testdict = split_fdict(fdict, test_size=0.2, random_state = 123)

In [8]:
trdict, valdict = split_fdict(trdict, test_size=0.2, random_state = 456)

In [9]:
valdict['clouds'][0:5]

['data/clouds/0678.jpeg',
 'data/clouds/0701.jpeg',
 'data/clouds/0431.jpeg',
 'data/clouds/0033.jpeg',
 'data/clouds/0290.jpeg']

### Copy images files into temp directories

In [10]:
import tempfile
import shutil

In [11]:
tmp_train_dir = tempfile.TemporaryDirectory()
tmp_valid_dir = tempfile.TemporaryDirectory()
tmp_test_dir = tempfile.TemporaryDirectory()

In [12]:
def copy_images(tmp_dir, data_dict):
    for cat in data_dict.keys():
        os.makedirs("{}/{}".format(tmp_dir.name, cat), exist_ok=True)
        for img_path in data_dict[cat]:
            img_name = img_path.split("/")[-1]
            shutil.copy2(img_path, "{}/{}/{}".format(tmp_dir.name, cat, img_name))

In [13]:
%%time
copy_images(tmp_train_dir, trdict)

CPU times: user 1.09 s, sys: 5.19 s, total: 6.28 s
Wall time: 6.28 s


In [14]:
%%time
copy_images(tmp_valid_dir, valdict)

CPU times: user 320 ms, sys: 1.19 s, total: 1.51 s
Wall time: 1.51 s


In [15]:
%%time
copy_images(tmp_test_dir, testdict)

CPU times: user 332 ms, sys: 1.53 s, total: 1.86 s
Wall time: 2.14 s


### Set up data generator

In [16]:
from keras.preprocessing.image import ImageDataGenerator

In [17]:
IMG_SIZE = 256
BATCH_SIZE = 32

In [18]:
TRAIN_DATAGEN = ImageDataGenerator(
        rescale=1./255,
)

TRAIN_GENERATOR = TRAIN_DATAGEN.flow_from_directory(
        directory=tmp_train_dir.name,
        target_size=(IMG_SIZE, IMG_SIZE),
        class_mode='sparse',
        batch_size=BATCH_SIZE,
)

Found 7489 images belonging to 16 classes.


In [19]:
VALID_DATAGEN = ImageDataGenerator(
        rescale=1./255,
)

VALID_GENERATOR = VALID_DATAGEN.flow_from_directory(
        directory=tmp_valid_dir.name,
        target_size=(IMG_SIZE, IMG_SIZE),
        class_mode='sparse',
        batch_size=BATCH_SIZE,
)

Found 1880 images belonging to 16 classes.


In [20]:
TEST_DATAGEN = ImageDataGenerator(
        rescale=1./255,
)

TEST_GENERATOR = TEST_DATAGEN.flow_from_directory(
        directory=tmp_test_dir.name,
        target_size=(IMG_SIZE, IMG_SIZE),
        class_mode='sparse',
        batch_size=1,
)

Found 2352 images belonging to 16 classes.


## Model training and evaluation

In [21]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers

In [22]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(TRAIN_GENERATOR.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [23]:
for layer in model.layers[:len(base_model.layers)]:
    layer.trainable = False
for layer in model.layers[len(base_model.layers):]:
    layer.trainable = True

In [24]:
optimizer = optimizers.Adam(lr=0.001, decay=0.01)

In [25]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=["accuracy"])

### Training

In [26]:
model.fit_generator(
    generator=TRAIN_GENERATOR
    , steps_per_epoch=TRAIN_GENERATOR.n // BATCH_SIZE 
    , epochs=5
    , verbose=1
    , validation_data=VALID_GENERATOR
    , validation_steps=VALID_GENERATOR.n // BATCH_SIZE
)

Epoch 1/5
 13/234 [>.............................] - ETA: 6:09 - loss: 2.6347 - acc: 0.3053

/usr/local/lib/python3.5/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


115/234 [=============>................] - ETA: 3:31 - loss: 1.4252 - acc: 0.5603

/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2496: DecompressionBombWarning: Image size (103926732 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


131/234 [===============>..............] - ETA: 3:04 - loss: 1.3806 - acc: 0.5713

/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2496: DecompressionBombWarning: Image size (107712000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


233/234 [============================>.] - ETA: 1s - loss: 1.1905 - acc: 0.6194

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1572864 bytes but only got 0. Skipping tag 42
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19600506880 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2684485632 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2751528960 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expe

234/234 [==============================] - 538s 2s/step - loss: 1.1880 - acc: 0.6201 - val_loss: 0.8465 - val_acc: 0.7328
Epoch 2/5
234/234 [==============================] - 533s 2s/step - loss: 0.7966 - acc: 0.7321 - val_loss: 0.7629 - val_acc: 0.7301
Epoch 3/5
234/234 [==============================] - 522s 2s/step - loss: 0.7381 - acc: 0.7440 - val_loss: 0.7850 - val_acc: 0.7177
Epoch 4/5
234/234 [==============================] - 534s 2s/step - loss: 0.6767 - acc: 0.7643 - val_loss: 0.7335 - val_acc: 0.7338
Epoch 5/5
234/234 [==============================] - 532s 2s/step - loss: 0.6528 - acc: 0.7737 - val_loss: 0.7875 - val_acc: 0.7177


### Save the model

In [28]:
import json

In [31]:
model.save_weights('{}/simple-classification.h5'.format(SAVE_MODEL_PATH))
with open("{}/simple-classification.json".format(SAVE_MODEL_PATH), 'w') as f:
    json.dump(json.loads(model.to_json()), f) # model.to_json() is a STRING of json
with open("{}/simple-classification-labels.json".format(SAVE_MODEL_PATH), 'w') as f:
    json.dump(TRAIN_GENERATOR.class_indices, f)

### Evaluation

In [32]:
%%time

model.evaluate_generator(
    TEST_GENERATOR
    , steps=TEST_GENERATOR.n
)

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1441814 bytes but only got 0. Skipping tag 33
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 428 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 196608 bytes but only got 0. Skipping tag 60
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 196608 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5

CPU times: user 6min 51s, sys: 26.3 s, total: 7min 17s
Wall time: 3min 49s


[0.74920513820500223, 0.72406462585034015]

left: loss, right: accuracy